In [1]:
import sys
sys.path.insert(0, r'/Users/rvv/Github/BGlib/')
sys.path.insert(0, r'/Users/rvv/Github/sidpy/')

import BGlib.be as belib
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
import sidpy
import SciFiReaders as sr

In [2]:
folder_path = r'/Users/rvv/Dropbox (ORNL)/PTO LSMO JCY/031822'
file_name = r'60x60_0deg_0004.h5'
path_to_file = os.path.join(folder_path, file_name)

In [3]:
path_gwy = r'/Users/rvv/Dropbox (ORNL)/AE Related Stuff/RL for Walls/Paper/Resubmission/12202023/PTO_110_Virgin0001.gwy'
gwy_reader = sr.GwyddionReader(path_gwy)
data = gwy_reader.read()

In [4]:
data[0]

#threadpoolctl version. We need to insist on version 3.2.0 


sidpy.Dataset of type IMAGE with:
 dask.array<array, shape=(256, 256), dtype=float64, chunksize=(256, 256), chunktype=numpy.ndarray>
 data contains: HeightRetrace (m)
 and Dimensions: 
x:  x (m) of size (256,)
y:  y (m) of size (256,)

In [5]:
patcher = belib.translators.LabViewH5Patcher()
patcher.translate(path_to_file)

reader = sr.Usid_reader(path_to_file)
data = reader.read()

/Users/rvv/Github/sidpy/sidpy/sid/translator.py:42: FutureWarning: Consider using sidpy.Reader instead of sidpy.Translator if possible and contribute your reader to ScopeReaders
  warn('Consider using sidpy.Reader instead of sidpy.Translator if '


File is already Pycroscopy ready.


In [6]:
data

sidpy.Dataset of type UNKNOWN with:
 dask.array<array, shape=(60, 60, 123, 64, 2, 2), dtype=complex64, chunksize=(45, 45, 45, 45, 2, 2), chunktype=numpy.ndarray>
 data contains: quantity (a.u.)
 and Dimensions: 
X:  X (nm) of size (60,)
Y:  Y (nm) of size (60,)
Frequency:  Frequency (Hz) of size (123,)
DC_Offset:  DC_Offset (V) of size (64,)
Field:  Field (generic) of size (2,)
Cycle:  Cycle (generic) of size (2,)

In [7]:
beps_raw = data

freq_axis = beps_raw.labels.index('Frequency (Hz)')
freq_vec = beps_raw._axes[freq_axis].values
all_dims = np.arange(len(data.shape))
ind_dims = np.delete(all_dims, freq_axis)
print(ind_dims, freq_vec.shape)


[0 1 3 4 5] (123,)


In [8]:
#Do some SHO Fitting
import sidpy
from BGlib.be.analysis.utils.sidpy_sho_fitter import SHOestimateGuess, SHOestimateGuess, SHO_fit_flattened

                            

In [15]:
lb = [1E-6, freq_vec.min(), 50, -2*np.pi]
ub = [1E-3, freq_vec.max(), 500, 2*np.pi]
beps_small = beps_raw[:5, :5, :]

fitter = sidpy.proc.fitter.SidFitter(beps_small, SHO_fit_flattened,num_workers=8,
                                     guess_fn = SHOestimateGuess,ind_dims=ind_dims,
                           threads=1, return_cov=False, return_fit=True, return_std=False,
                           km_guess=True,num_fit_parms = 4, n_clus = 5)


In [16]:
fitter.do_guess()
#fit_dset = fitter.do_fit(bounds = (lb,ub))

In [18]:
output = fitter.do_fit(bounds = (lb,ub))

/Users/rvv/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


---Finished KMeans, onto fiting each KM Center---
Fitting center 0
Fitting center 1
Fitting center 2
Fitting center 3
Fitting center 4
Shapes of output of fitting function is 246 and original data is 123                   Reshaping output dataset. You are responsible for reshaping


In [19]:
#Let's check the fitted dataset.
output

[sidpy.Dataset of type IMAGE_STACK with:
  dask.array<transpose, shape=(5, 5, 64, 2, 2, 4), dtype=float64, chunksize=(5, 5, 64, 2, 2, 4), chunktype=numpy.ndarray>
  data contains: generic (generic)
  and Dimensions: 
 X:  X (nm) of size (5,)
 Y:  Y (nm) of size (5,)
 DC_Offset:  DC_Offset (V) of size (64,)
 Field:  Field (generic) of size (2,)
 Cycle:  Cycle (generic) of size (2,)
 fit_parms:  fit_parameters (a.u.) of size (4,)
  with metadata: ['fit_parms_dict'],
 sidpy.Dataset of type UNKNOWN with:
  dask.array<array, shape=(6400, 246), dtype=float64, chunksize=(6400, 246), chunktype=numpy.ndarray>
  data contains: quantity (a.u.)
  and Dimensions: 
 X:  X (nm) of size (6400,)
 Y:  Y (nm) of size (246,)
  with metadata: ['fit_parms_dict']]

In [31]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Create some sample 3D data
x = np.linspace(0, 10, 100)
y = np.linspace(0, 5, 50)
freq = np.linspace(1, 10, 100)
data = np.zeros(shape=(len(x), len(y), len(freq)))

for indx in range(data.shape[0]):
    for indy in range(data.shape[1]):
        xval = x[indx]
        yval = y[indy]
        data[indx, indy,:] = np.sin(xval) * np.cos(yval) * np.sin(freq)

# Create initial image plot and spectrum plot
fig, (ax_image, ax_spectrum) = plt.subplots(1, 2, figsize=(10, 5))
image = ax_image.imshow(data[:, :, 0], extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', aspect='auto')
ax_image.set_title('Spatial Image')
ax_image.set_xlabel('X-axis')
ax_image.set_ylabel('Y-axis')

# Use a 2D array for the spectrum data
spectrum_line, = ax_spectrum.plot(freq, data[0, 0, :], label='Spectrum')
ax_spectrum.set_title('Spectrum at selected point')
ax_spectrum.set_xlabel('Frequency')
ax_spectrum.set_ylabel('Amplitude')
ax_spectrum.legend()

# Create widgets for interaction
slider = widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description='Amplitude:')
checkbox = widgets.Checkbox(value=False, description='Show Grid')

# Function to update the plot based on widget values
def update_plot(change):
    spectrum_line.set_ydata(np.sin(change.new * x))
    ax_spectrum.set_title(f'Spectrum at selected point (Amplitude={change.new:.1f})')
    ax_spectrum.grid(checkbox.value)
    fig.canvas.draw()

# Function to update the spectrum plot based on selected point
def update_spectrum_plot(event):
    if event.inaxes == ax_image:
        x_click, y_click = event.xdata, event.ydata
        x_index = np.argmin(np.abs(x - x_click))
        y_index = np.argmin(np.abs(y - y_click))
        spectrum_line.set_ydata(data[y_index, x_index, :])
        ax_spectrum.set_title(f'Spectrum at ({x_click:.2f}, {y_click:.2f})')
        fig.canvas.draw()

# Attach the update functions to the widget and the figure's click event
slider.observe(update_plot, 'value')
checkbox.observe(update_plot, 'value')
fig.canvas.mpl_connect('button_press_event', update_spectrum_plot)

# Display the widgets and plots
display(widgets.VBox([slider, checkbox]), fig)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
#We need to write a visualizer that will be capable of handling the above data and plot the fits along with it.
#We need drop downs for cycle, field, and fit parameter. We can also determine the R^2 and plot that if requested...

data.shape

(50, 100)